In [8]:
import os
import json
import uuid
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

# 환경 변수 설정
# os.environ['PINECONE_API_KEY'] = 'your_actual_pinecone_api_key_here'
api_key = os.getenv('PINECONE_API_KEY')
if not api_key:
    raise ValueError("Pinecone API key is not set. Please ensure the PINECONE_API_KEY environment variable is correct.")

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=api_key)

# 클라우드 및 리전 설정
cloud = os.getenv('PINECONE_CLOUD', 'aws')
region = os.getenv('PINECONE_REGION', 'us-east-1')
spec = ServerlessSpec(cloud=cloud, region=region)

# 인덱스 이름
index_name = 'example-index'

# 인덱스 확인 및 재생성
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# 새 인덱스 생성
pc.create_index(
    index_name,
    dimension=384,  # 'paraphrase-multilingual-MiniLM-L12-v2'의 차원은 384
    metric='dotproduct',
    spec=spec
)

# 인덱스 객체 얻기
index = pc.Index(index_name)

# 모델 로드
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# JSON 파일 로드 및 데이터 처리
with open('output_tables/(1) 업무기준.json', 'r') as f:
    data = json.load(f)
    for table_name, rows in data.items():
        for row in rows:
            # 텍스트를 벡터로 변환
            vector = model.encode(row['text'])

            # 메타데이터 준비
            metadata = row['metadata']

            # 고유 ID 생성
            vector_id = str(uuid.uuid4())

            # Pinecone에 데이터 업로드
            index.upsert(vectors=[(vector_id, vector, metadata)])

# 필요한 경우 Pinecone 인덱스를 종료
# pc.close()
